### D)
Extend this and try to see how much of the bedroom you can reconstruct from the RGB and depth images.
You can extend a point cloud by `new = source + target`. Remember to resample the point cloud after a view additions so it does not get too large: `down_source = source.voxel_down_sample(voxel_size=0.05)`

In [430]:
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
import copy

from tqdm import tqdm

import glob
import os

In [431]:
rgb_paths = glob.glob('RGBD/color/*.jpg')
depth_paths = glob.glob('RGBD/depth/*.png')

In [432]:
rgb_paths.sort(key=lambda path: int(os.path.basename(path).split('.')[0]))
depth_paths.sort(key=lambda path: int(os.path.basename(path).split('.')[0]))

In [433]:
# rgb_paths = rgb_paths[:100]
# depth_paths = depth_paths[:100]

In [434]:
print(rgb_paths)

['RGBD/color/000000.jpg', 'RGBD/color/000001.jpg', 'RGBD/color/000002.jpg', 'RGBD/color/000003.jpg', 'RGBD/color/000004.jpg', 'RGBD/color/000005.jpg', 'RGBD/color/000006.jpg', 'RGBD/color/000007.jpg', 'RGBD/color/000008.jpg', 'RGBD/color/000009.jpg', 'RGBD/color/000010.jpg', 'RGBD/color/000011.jpg', 'RGBD/color/000012.jpg', 'RGBD/color/000013.jpg', 'RGBD/color/000014.jpg', 'RGBD/color/000015.jpg', 'RGBD/color/000016.jpg', 'RGBD/color/000017.jpg', 'RGBD/color/000018.jpg', 'RGBD/color/000019.jpg', 'RGBD/color/000020.jpg', 'RGBD/color/000021.jpg', 'RGBD/color/000022.jpg', 'RGBD/color/000023.jpg', 'RGBD/color/000024.jpg', 'RGBD/color/000025.jpg', 'RGBD/color/000026.jpg', 'RGBD/color/000027.jpg', 'RGBD/color/000028.jpg', 'RGBD/color/000029.jpg', 'RGBD/color/000030.jpg', 'RGBD/color/000031.jpg', 'RGBD/color/000032.jpg', 'RGBD/color/000033.jpg', 'RGBD/color/000034.jpg', 'RGBD/color/000035.jpg', 'RGBD/color/000036.jpg', 'RGBD/color/000037.jpg', 'RGBD/color/000038.jpg', 'RGBD/color/000039.jpg',

In [435]:
print(len(rgb_paths))
print(len(depth_paths))

401
401


In [436]:
target = None
trans_init = None

In [437]:
estimation_method = o3d.pipelines.registration.TransformationEstimationPointToPoint()
threshold = 0.5
trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

camera = o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)

voxel_size = 0.01
voxel_size_2 = 0.05 
voxel_size_final = 0.005

In [438]:
point_cloud = o3d.geometry.PointCloud()

for i, (rgb_path, depth_path) in tqdm(enumerate(zip(rgb_paths, depth_paths)), total=len(rgb_paths)):

    rgb = o3d.io.read_image(rgb_path)
    depth = o3d.io.read_image(depth_path)
    
    # print(rgb_path)
    # print(depth_path)

    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgb, depth, convert_rgb_to_intensity=True)

    source = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image, camera)

    source = source.voxel_down_sample(voxel_size)

    if target is not None:
        target.estimate_normals()

        icp_result = o3d.pipelines.registration.registration_icp(
            source, target, threshold, trans_init,
            estimation_method)

        # print(icp_result)
        trans_init = icp_result.transformation

        source.transform(trans_init)

        # source.paint_uniform_color(
            # [1/len(rgb_paths) * (i+1), 1/len(rgb_paths) * (i+1), 1/len(rgb_paths) * (i+1)])

        point_cloud += source
        
    if i % 50 == 0:
        point_cloud = point_cloud.voxel_down_sample(voxel_size_2)

    target = source

    # point_cloud.transform([[1, 0, 0, 0],
    #                        [0, -1, 0, 0],
    #                        [0, 0, -1, 0], [0, 0, 0, 1]])

    # o3d.visualization.draw_geometries([point_cloud])

100%|██████████| 401/401 [02:34<00:00,  2.59it/s]


In [439]:
point_cloud.points

std::vector<Eigen::Vector3d> with 14185 elements.
Use numpy.asarray() to access data.

In [440]:
point_cloud.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

o3d.visualization.draw_geometries([point_cloud])